#Grouping data in pandas and pyspark

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
import pandas as pd 
df = pd.read_csv("https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv", keep_default_na=False)
sdf = spark.createDataFrame(df)
sdf.show()
df.head()

+-----------------+-------------+---------------+-------------+----------------------------+---------+
|          country|beer_servings|spirit_servings|wine_servings|total_litres_of_pure_alcohol|continent|
+-----------------+-------------+---------------+-------------+----------------------------+---------+
|      Afghanistan|            0|              0|            0|                         0.0|       AS|
|          Albania|           89|            132|           54|                         4.9|       EU|
|          Algeria|           25|              0|           14|                         0.7|       AF|
|          Andorra|          245|            138|          312|                        12.4|       EU|
|           Angola|          217|             57|           45|                         5.9|       AF|
|Antigua & Barbuda|          102|            128|           45|                         4.9|       NA|
|        Argentina|          193|             25|          221|          

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF


#Which continent drinks more beer on average?

In [0]:
df.groupby(by='continent').agg({"beer_servings":'mean'}).sort_values(by="beer_servings", ascending=False)

,beer_servings
continent,
EU,193.777778
SA,175.083333
NA,145.434783
OC,89.687500
AF,61.471698
AS,37.045455


In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
sdf.groupBy("continent").agg(mean("beer_servings").alias("avg_beer")).orderBy(["avg_beer"],ascending=False).withColumn('avgg', round('avg_beer',2)).show()

+---------+------------------+------+
|continent|          avg_beer|  avgg|
+---------+------------------+------+
|       EU|193.77777777777777|193.78|
|       SA|175.08333333333334|175.08|
|       NA|145.43478260869566|145.43|
|       OC|           89.6875| 89.69|
|       AF|61.471698113207545| 61.47|
|       AS| 37.04545454545455| 37.05|
+---------+------------------+------+



# For each continent print the statistics for wine consumption.

In [0]:
df.groupby('continent').wine_servings.describe()

,count,mean,std,min,25%,50%,75%,max
continent,,,,,,,,
AF,53.0,16.264151,38.846419,0.0,1.0,2.0,13.00,233.0
AS,44.0,9.068182,21.667034,0.0,0.0,1.0,8.00,123.0
EU,45.0,142.222222,97.421738,0.0,59.0,128.0,195.00,370.0
NA,23.0,24.521739,28.266378,1.0,5.0,11.0,34.00,100.0
OC,16.0,35.625000,64.555790,0.0,1.0,8.5,23.25,212.0
SA,12.0,62.416667,88.620189,1.0,3.0,12.0,98.50,221.0


In [0]:
sdf.groupBy('continent').agg(sum("wine_servings"), max("wine_servings")).show()

+---------+------------------+------------------+
|continent|sum(wine_servings)|max(wine_servings)|
+---------+------------------+------------------+
|       NA|               564|               100|
|       SA|               749|               221|
|       AS|               399|               123|
|       OC|               570|               212|
|       EU|              6400|               370|
|       AF|               862|               233|
+---------+------------------+------------------+



#Print the mean alcohol consumption per continent for every column



In [0]:
df.groupby('continent').mean()

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
continent,,,,
AF,61.471698,16.339623,16.264151,3.007547
AS,37.045455,60.840909,9.068182,2.170455
EU,193.777778,132.555556,142.222222,8.617778
NA,145.434783,165.739130,24.521739,5.995652
OC,89.687500,58.437500,35.625000,3.381250
SA,175.083333,114.750000,62.416667,6.308333


In [0]:
sdf.groupBy("continent").mean().show()

+---------+------------------+--------------------+------------------+---------------------------------+
|continent|avg(beer_servings)|avg(spirit_servings)|avg(wine_servings)|avg(total_litres_of_pure_alcohol)|
+---------+------------------+--------------------+------------------+---------------------------------+
|       NA|145.43478260869566|   165.7391304347826| 24.52173913043478|                5.995652173913044|
|       SA|175.08333333333334|              114.75|62.416666666666664|                6.308333333333334|
|       AS| 37.04545454545455|   60.84090909090909| 9.068181818181818|               2.1704545454545454|
|       OC|           89.6875|             58.4375|            35.625|                          3.38125|
|       EU|193.77777777777777|  132.55555555555554|142.22222222222223|                8.617777777777778|
|       AF|61.471698113207545|  16.339622641509433|16.264150943396228|               3.0075471698113208|
+---------+------------------+--------------------+----

#Print the median alcohol consumption per continent for every column

In [0]:
df.groupby("continent").median()

,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
continent,,,,
AF,32.0,3.0,2.0,2.30
AS,17.5,16.0,1.0,1.20
EU,219.0,122.0,128.0,10.00
NA,143.0,137.0,11.0,6.30
OC,52.5,37.0,8.5,1.75
SA,162.5,108.5,12.0,6.85


In [0]:

sdf.groupBy("continent").agg(\
    expr("percentile(wine_servings, array(0.5))").alias("median_wine"),\
        expr("percentile(beer_servings, array(0.5))").alias("median_beer"),\
            expr("percentile(spirit_servings, array(0.5))").alias("median_spirit"),\
                expr("percentile(total_litres_of_pure_alcohol, array(0.5))").alias("median_total"),\
    ).show()

+---------+-----------+-----------+-------------+------------+
|continent|median_wine|median_beer|median_spirit|median_total|
+---------+-----------+-----------+-------------+------------+
|       NA|     [11.0]|    [143.0]|      [137.0]|       [6.3]|
|       SA|     [12.0]|    [162.5]|      [108.5]|      [6.85]|
|       AS|      [1.0]|     [17.5]|       [16.0]|       [1.2]|
|       OC|      [8.5]|     [52.5]|       [37.0]|      [1.75]|
|       EU|    [128.0]|    [219.0]|      [122.0]|      [10.0]|
|       AF|      [2.0]|     [32.0]|        [3.0]|       [2.3]|
+---------+-----------+-----------+-------------+------------+



#Print the mean, min and max values for spirit consumption.

In [0]:
print(df.groupby('continent').spirit_servings.describe())
df.groupby('continent').spirit_servings.agg(["mean","max","min"])

           count        mean        std   min     25%    50%     75%    max
continent                                                                  
AF          53.0   16.339623  28.102794   0.0    1.00    3.0   19.00  152.0
AS          44.0   60.840909  84.362160   0.0    1.00   16.0   98.00  326.0
EU          45.0  132.555556  77.589115   0.0   81.00  122.0  173.00  373.0
NA          23.0  165.739130  94.993884  68.0  101.00  137.0  190.50  438.0
OC          16.0   58.437500  70.504817   0.0   18.00   37.0   65.25  254.0
SA          12.0  114.750000  77.077440  25.0   65.75  108.5  148.75  302.0


,mean,max,min
continent,,,
AF,16.339623,152,0
AS,60.840909,326,0
EU,132.555556,373,0
NA,165.739130,438,68
OC,58.437500,254,0
SA,114.750000,302,25


In [0]:
sdf.groupBy("continent").agg(mean("spirit_servings"),max("spirit_servings"),min("spirit_servings")).show()

+---------+--------------------+--------------------+--------------------+
|continent|avg(spirit_servings)|max(spirit_servings)|min(spirit_servings)|
+---------+--------------------+--------------------+--------------------+
|       NA|   165.7391304347826|                 438|                  68|
|       SA|              114.75|                 302|                  25|
|       AS|   60.84090909090909|                 326|                   0|
|       OC|             58.4375|                 254|                   0|
|       EU|  132.55555555555554|                 373|                   0|
|       AF|  16.339622641509433|                 152|                   0|
+---------+--------------------+--------------------+--------------------+

